In [51]:
import random
import praw
from elasticsearch import Elasticsearch
from elasticsearch import helpers
scan = helpers.scan

## Setup Reddit API and ElasticSearch Connection

In [52]:
reddit = praw.Reddit(client_id='Cz8OU1vxajnWDw',
                     client_secret='5qax29ZPI2_Rdjc1TsXXEypFduk',
                     redirect_uri='http://localhost:8080',
                     user_agent='my user agent')

print(reddit.auth.url(['identity'], '...', 'permanent'))

es = Elasticsearch([{'host':'localhost','port':9200}])
es

https://www.reddit.com/api/v1/authorize?client_id=Cz8OU1vxajnWDw&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [54]:
src_results = es.search(scroll='360m',      
    body =
    {
      "query": {
        "function_score": {
          "query": {
            "bool": {
              "must": [
                {
                  "bool": {
                    "must_not": [
                      {
                        "exists": {
                          "field": "commenters"
                        }
                      }
                    ]
                  }
                },
                {
                  "exists": {
                    "field": "bm.overall"
                  }
                }
              ]
            }
          },
          "random_score": {
            "seed": random.randint(1, 1000)
          },
          "boost_mode": "replace"
        }
      }
    }, index = 'redditors', size = 2000)

for src_doc in src_results['hits']['hits']:
    redditor = reddit.redditor(src_doc['_id'])
    commenters = set()
    
    #print(redditor.name)
    try:
        for submission in redditor.submissions.hot():
            for comment in submission.comments:
                try:
                    if comment.author is not None and comment.author != redditor.name:
                        commenters.add(str(comment.author))
                except:
                    pass
    except:
        print("Forbidden: " + redditor.name)
        pass
    
    update = {
        "doc": {
            "commenters": list(commenters)
        }
    }
    #print(len(commenters))
    es.update(index = 'redditors', id = src_doc['_id'], body = update)
    



Forbidden: TheWoahism
Forbidden: five_focus
Forbidden: merc_witha_mouth
Forbidden: Tonkinese-Dream
Forbidden: EthenScottwalkerFC
Forbidden: Katana_Viking
Forbidden: jimbotexas
Forbidden: WowbaggerBowerick
Forbidden: Thoragaverung
Forbidden: retakeomahabeach1988
Forbidden: Why_Keep_Coming_Back
Forbidden: Throwawaybusteddream
Forbidden: Fisabilillah_124
Forbidden: JoJoAbrams1972
Forbidden: tmig145
Forbidden: kubrickfan1234
Forbidden: Visitor18
Forbidden: karmakutti715
Forbidden: MFMyLiver
Forbidden: wff84
Forbidden: ApartWin
Forbidden: SmarterGhoul
Forbidden: GreenBastard1133
Forbidden: ck_9900
Forbidden: ohhh-caitlyn
Forbidden: minecraftdude2006
Forbidden: R3troville
Forbidden: ConesofDunshire9999
Forbidden: maxfield13
Forbidden: johnnysleep
Forbidden: UnimaginablePotato
Forbidden: Shade_of_grey_
Forbidden: vitriolic_bitch
Forbidden: NJane_
Forbidden: OfficialGoneZero
Forbidden: BattyWiseau
Forbidden: darashorten
Forbidden: Lewd_Otaku
Forbidden: Quetzalcoatllll
Forbidden: joff50105
Forb